In [103]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame
from datetime import datetime

In [47]:
careplans = pd.read_csv('careplans.csv')
immunizations = pd.read_csv('immunizations.csv')
patients = pd.read_csv('patients.csv')
observations = pd.read_csv('observations.csv')
encounters = pd.read_csv('encounters.csv')
conditions = pd.read_csv('conditions.csv')
patients = pd.read_csv('patients.csv')
procedures = pd.read_csv('procedures.csv')
medications = pd.read_csv('medications.csv')  
covid_status = pd.read_csv('COVID_19 Status submission.csv')
hospitalized = pd.read_csv('hospitalized.csv')
ventilation = pd.read_csv('ventilation_status.csv')
days_in_icu = pd.read_csv('days_in_icu.csv')
test = pd.read_csv('/kaggle/output/kaggle/working/test_patients.csv')

## Exploratory Data Analysis and Preprocessing

Identify death in 2020 

In [6]:
patients['DEATHDATE']= patients['DEATHDATE'].astype('datetime64[ns]')
patients['BIRTHDATE']= patients['BIRTHDATE'].astype('datetime64[ns]')
patients['Decease_2020'] = 0
patients.loc[patients.DEATHDATE.dt.year == 2020,'Decease_2020'] = 1 
patients.loc[patients.DEATHDATE.dt.year < 2020,'Decease_2020'] = 0.1

Compute age 

In [10]:
patients['DEATHDATE'] = patients['DEATHDATE'].fillna(np.datetime64('NOW'))
patients['AGE'] = (patients['DEATHDATE'] - patients['BIRTHDATE'].astype('M8[D]')).astype('timedelta64[Y]').astype(float)

Compute num of total hospitalized days for each patients 

In [11]:
from datetime import datetime
encounters['START']= encounters['START'].astype('datetime64[ns]')
encounters['STOP']= encounters['STOP'].astype('datetime64[ns]')
total_hospitalized_days = DataFrame(encounters[['PATIENT','START','STOP']])
total_hospitalized_days['NUM_DAYS_HOPS'] = (total_hospitalized_days['STOP']-total_hospitalized_days['START']).astype('timedelta64[D]').astype(float)
total_hospitalized_days = total_hospitalized_days.groupby(['PATIENT']).sum()
patients = pd.merge(patients,
                   total_hospitalized_days,
                   how = 'left',
                   left_on = 'Id',
                   right_on= 'PATIENT')
patients['NUM_DAYS_HOPS'] = patients['NUM_DAYS_HOPS'].fillna(0.0)

First turn feature selection 

In [13]:
patients['Marital_STAT'] = 0
patients.loc[patients.MARITAL == 'M','Marital_STAT'] = 1
patients = patients.drop(['BIRTHDATE','DEATHDATE','SSN','FIRST','LAST','MAIDEN','PREFIX'],axis = 1)
patients = patients.drop(['ADDRESS'],axis = 1)
patients = patients.drop(['BIRTHPLACE', 'CITY', 'STATE', 'COUNTY', 'ZIP'],axis = 1)

,Id,DRIVERS,PASSPORT,SUFFIX,MARITAL,RACE,ETHNICITY,GENDER,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,Decease_2020,COVID_STAT,AGE,NUM_DAYS_HOPS,Marital_STAT
0,8c6f61a6-1beb-4b51-980e-381722ca7e33,NaN,NaN,NaN,NaN,white,nonhispanic,M,42.500733,-70.993119,37746.85,591.60,0.1,1,1.0,0.0,0
1,6732e6fc-f8f3-4b1a-924a-47f68f19526c,S99982468,X29369501X,NaN,M,white,nonhispanic,F,42.328148,-71.792040,151885.57,9692.77,0.0,1,48.0,0.0,1
2,4772a573-1fb1-49ec-be86-dc2c4b176052,S99966580,X1644093X,NaN,M,white,nonhispanic,M,42.693834,-71.163903,1546830.50,15857.37,0.0,1,82.0,0.0,1


Only consider about died patients in 2020.

In [15]:
target_patients = DataFrame(patients[patients.Decease_2020 != 0.1])

(21479, 17)

## Feature selection from other related datasets

In [16]:
observations['DATE'] = observations['DATE'].astype('datetime64[ns]')
observations = observations.loc[observations.DATE.dt.year == 2020]
observations1 = DataFrame(observations[observations.VALUE =='Non-small cell lung cancer (disorder)'])
observations2 = DataFrame(observations[observations.VALUE =='Neoplasm of prostate'])
observations3 = DataFrame(observations[observations.VALUE =='Pneumonia'])
observations4 = DataFrame(observations[observations.VALUE =='Myocardial Infarction'])
observations5 = DataFrame(observations[observations.VALUE =='Small cell carcinoma of lung (disorder)'])
target_patients['Non_small_cell_lung_cancer'] = 0
target_patients.loc[target_patients.Id.isin(observations1.PATIENT),'Non_small_cell_lung_cancer'] = 1
target_patients['Neoplasm_of_prostate'] = 0
target_patients.loc[target_patients.Id.isin(observations2.PATIENT),'Neoplasm_of_prostate'] = 1
target_patients['Pneumonia'] = 0
target_patients.loc[target_patients.Id.isin(observations3.PATIENT),'Pneumonia'] = 1                                          #!! 
target_patients['Myocardial_Infarction'] = 0
target_patients.loc[target_patients.Id.isin(observations4.PATIENT),'Myocardial_Infarction'] = 1
target_patients['Small_cell_carcinoma_of_lung'] = 0
target_patients.loc[target_patients.Id.isin(observations5.PATIENT),'Small_cell_carcinoma_of_lung'] = 1

In [17]:
encounters1 = DataFrame(encounters[encounters.ENCOUNTERCLASS == 'emergency'])
encounters2 = DataFrame(encounters[encounters.ENCOUNTERCLASS == 'ambulatory'])
encounters1 = encounters1[encounters1.REASONDESCRIPTION =='Chronic congestive heart failure (disorder)']
encounters2 = encounters2[encounters2.REASONDESCRIPTION =='Chronic congestive heart failure (disorder)']
encounters1.loc[encounters1.START.dt.year == 2020]
encounters2.loc[encounters2.START.dt.year == 2020]
target_patients['Chronic_congestive_heart_failure'] = 0
target_patients.loc[target_patients.Id.isin(encounters1.PATIENT),'Chronic_congestive_heart_failure'] = 1
target_patients.loc[target_patients.Id.isin(encounters2.PATIENT),'Chronic_congestive_heart_failure'] = 1

In [18]:
procedures1 = procedures[procedures.DESCRIPTION =='Oxygen Therapy']
procedures11 = procedures[procedures.DESCRIPTION =='Oxygen administration by mask (procedure)']
target_patients['Oxygen_Therapy'] = 0
target_patients.loc[target_patients.Id.isin(procedures1.PATIENT),'Oxygen_Therapy'] = 1
target_patients.loc[target_patients.Id.isin(procedures11.PATIENT),'Oxygen_Therapy'] = 1

procedures2 = procedures[procedures.DESCRIPTION =='Plain chest X-ray (procedure)']
procedures22 = procedures[procedures.DESCRIPTION =='Plain chest X-ray']
target_patients['Plain_chest_X_ray'] = 0
target_patients.loc[target_patients.Id.isin(procedures2.PATIENT),'Plain_chest_X_ray'] = 1
target_patients.loc[target_patients.Id.isin(procedures22.PATIENT),'Plain_chest_X_ray'] = 1

procedures3 = procedures[procedures.DESCRIPTION =='Mental health screening (procedure)']
target_patients['Mental_health_screening'] = 0
target_patients.loc[target_patients.Id.isin(procedures3.PATIENT),'Mental_health_screening'] = 1

procedures4 = procedures[procedures.REASONDESCRIPTION =='Non-small cell carcinoma of lung  TNM stage 1 (disorder)']
procedures44 = procedures[procedures.REASONDESCRIPTION =='Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)']
procedures444 = procedures[procedures.REASONDESCRIPTION =='Suspected lung cancer (situation)']
procedures4444 = procedures[procedures.REASONDESCRIPTION =='Pulmonary emphysema (disorder)']
target_patients['Pulmonary_diseases_cancer'] = 0
target_patients.loc[target_patients.Id.isin(procedures4.PATIENT),'Pulmonary_diseases_cancer'] = 1
target_patients.loc[target_patients.Id.isin(procedures44.PATIENT),'Pulmonary_diseases_cancer'] = 1
target_patients.loc[target_patients.Id.isin(procedures444.PATIENT),'Pulmonary_diseases_cancer'] = 1
target_patients.loc[target_patients.Id.isin(procedures4444.PATIENT),'Pulmonary_diseases_cancer'] = 1

procedures5 = procedures[procedures.REASONDESCRIPTION =='Chronic obstructive bronchitis (disorder)']
target_patients['Chronic_bronchitis'] = 0
target_patients.loc[target_patients.Id.isin(procedures5.PATIENT),'Chronic_bronchitis'] = 1

procedures6 = procedures[procedures.REASONDESCRIPTION =='Myocardial Infarction']
procedures66 = procedures[procedures.REASONDESCRIPTION =='Atrial Fibrillation']
target_patients['cardial_Atrial_diseases'] = 0
target_patients.loc[target_patients.Id.isin(procedures6.PATIENT),'cardial_Atrial_diseases'] = 1
target_patients.loc[target_patients.Id.isin(procedures66.PATIENT),'cardial_Atrial_diseases'] = 1

 
procedures7 = procedures[procedures.REASONDESCRIPTION =='Streptococcal sore throat (disorder)']
procedures77 = procedures[procedures.REASONDESCRIPTION =='Acute viral pharyngitis (disorder)']
target_patients['throat_pharyngitis'] = 0
target_patients.loc[target_patients.Id.isin(procedures7.PATIENT),'throat_pharyngitis'] = 1
target_patients.loc[target_patients.Id.isin(procedures77.PATIENT),'throat_pharyngitis'] = 1
 

In [19]:
immunizations1 = immunizations[immunizations.DESCRIPTION =='pneumococcal polysaccharide vaccine  23 valent']
immunizations2 = immunizations[immunizations.DESCRIPTION =='Pneumococcal conjugate PCV 13']
target_patients['immu_PCV'] = 0
target_patients.loc[target_patients.Id.isin(immunizations1.PATIENT),'immu_PCV'] = 1
target_patients.loc[target_patients.Id.isin(immunizations2.PATIENT),'immu_PCV'] = 1

In [20]:
conditions['START'] = conditions['START'].astype('datetime64[ns]')
conditions['STOP'] = conditions['STOP'].astype('datetime64[ns]')
conditions_curr = DataFrame(conditions[conditions.STOP.dt.year == 2020])
conditions1 = conditions[conditions.DESCRIPTION =='Respiratory distress (finding)']
conditions2 = conditions[conditions.DESCRIPTION =='Pneumonia (disorder)']
conditions3 = conditions[conditions.DESCRIPTION =='Hypoxemia (disorder)']                            
target_patients['Respiratory_distress'] = 0
target_patients.loc[target_patients.Id.isin(conditions1.PATIENT),'Respiratory_distress'] = 1
target_patients['Pneumonia_'] = 0
target_patients.loc[target_patients.Id.isin(conditions2.PATIENT),'Pneumonia_'] = 1
target_patients['Hypoxemia'] = 0
target_patients.loc[target_patients.Id.isin(conditions3.PATIENT),'Hypoxemia'] = 1

In [21]:
medications['START'] = medications['START'].astype('datetime64[ns]')
medications['STOP'] = medications['STOP'].astype('datetime64[ns]')
medications = DataFrame(medications[medications.STOP.dt.year == 2020])
medications1 = medications[medications.REASONDESCRIPTION =='Hypertension']
target_patients['Hypertension'] = 0
target_patients.loc[target_patients.Id.isin(medications1.PATIENT),'Hypertension'] = 1

medications2 = medications[medications.REASONDESCRIPTION =='Non-small cell carcinoma of lung  TNM stage 1 (disorder)']
medications22 = medications[medications.REASONDESCRIPTION =='Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)']  
target_patients['carcinoma_of_lung'] = 0
target_patients.loc[target_patients.Id.isin(medications2.PATIENT),'carcinoma_of_lung'] = 1
target_patients.loc[target_patients.Id.isin(medications22.PATIENT),'carcinoma_of_lung'] = 1

In [22]:
careplans1 = careplans[careplans.DESCRIPTION =='Respiratory therapy'] 
target_patients['Respiratory_therapy'] = 0
target_patients.loc[target_patients.Id.isin(careplans1.PATIENT),'Respiratory_therapy'] = 1

careplans2 = careplans[careplans.DESCRIPTION =='Chronic obstructive pulmonary disease clinical management plan'] 
target_patients['Chronic_pulmonary'] = 0
target_patients.loc[target_patients.Id.isin(careplans2.PATIENT),'Chronic_pulmonary'] = 1

careplans3 = careplans[careplans.DESCRIPTION =='Asthma self management'] 
target_patients['Asthma'] = 0
target_patients.loc[target_patients.Id.isin(careplans3.PATIENT),'Asthma'] = 1
 
careplans4 = careplans[careplans.DESCRIPTION =='Cancer care plan'] 
target_patients['Cancer_care_plan'] = 0
target_patients.loc[target_patients.Id.isin(careplans4.PATIENT),'Cancer_care_plan'] = 1

Remove useless features 

In [ ]:
target_patients['Gender'] = 0
target_patients.loc[target_patients.GENDER == 'M', 'Gender'] = 1
target_patients.drop(['GENDER'], axis = 1, inplace = True)
target_patients.drop(['DRIVERS','PASSPORT','SUFFIX','ETHNICITY','RACE','LAT','LON'], axis = 1, inplace = True)
target_patients.drop(['MARITAL'], axis = 1, inplace = True)
target_patients.set_index('Id', inplace=True)

## Prepare for final prediciton

Merge test csv with predictions from other parts  

In [76]:
days_in_icu = days_in_icu.rename(columns={"8c6f61a6-1beb-4b51-980e-381722ca7e33": "Id", "0.0": "Covid_NUM_DAYS_ICU"})
days_in_icu.set_index('Id', inplace=True)
covid_status = covid_status.rename(columns={ "predicted_proba": "COVID_STAT"})
covid_status.set_index('Id', inplace=True)
hospitalized = hospitalized.rename(columns={ "days_hospitalized": "Covid_NUM_DAYS_HOPS","PATIENT": "Id"})
hospitalized.set_index('Id', inplace=True)
ventilation = ventilation.rename(columns={"6732e6fc-f8f3-4b1a-924a-47f68f19526c": "Id", "0.0": "VENT"})
ventilation.set_index('Id', inplace=True)
test_patients = pd.merge(target_patients,
                        days_in_icu,
                        how = 'left',
                        on = 'Id')
test_patients = pd.merge(test_patients,
                        covid_status,
                        how = 'left',
                        on = 'Id')
test_patients = pd.merge(test_patients,
                        hospitalized,
                        how = 'left',
                        on = 'Id')
test_patients = pd.merge(test_patients,
                        ventilation,
                        how = 'left',
                        on = 'Id')
test_patients['Covid_NUM_DAYS_ICU'] = test_patients['Covid_NUM_DAYS_ICU'].fillna(0.0)
test_patients['VENT'] = test_patients['VENT'].fillna(0.0)
test_patients['COVID_STAT'] = test_patients['COVID_STAT'].fillna(0.0)
test_patients['Covid_NUM_DAYS_HOPS'] = test_patients['Covid_NUM_DAYS_HOPS'].fillna(0.0)
test = test_patients.to_csv('test_patients.csv')

After prediction, construct submission csv. 

In [ ]:
predict1 = pd.read_csv('/kaggle/input/final/Alive_Decease_status.csv')
predict2 = pd.read_csv('/kaggle/input/final/Alive_Decease_status2.csv')
predict1 = predict1.rename(columns={'1': 'Id'})
predict1.set_index('3', inplace=True)
predict1.drop(['0'], axis = 1, inplace = True )
predict1 = predict2.rename(columns={'1': 'Alive_Decease_status'})
predict1.to_csv('Alive_Decease_status2.csv')